Общая информация.

В тестовом задании 2 практических задания. 
Для рассмотрения вашей кандидатуры необходимо выполнить оба практических задания. 


1. Вам нужно определить собственные движения цены фьючерса ETHUSDT, исключив из них движения вызванные влиянием цены BTCUSDT. Опишите, какую методику вы выбрали, какие параметры подобрали, и почему.

2. Напишите программу на Python, которая в реальном времени (с минимальной задержкой) следит за ценой фьючерса ETHUSDT и используя выбранный вами метод, определяет собственные движение цены ETH. При изменении цены на 1% за последние 60 минут, программа выводит сообщение в консоль. При этом программа должна продолжать работать дальше, постоянно считывая актуальную цену.



api coincap --  https://coincap.io/api-key

docs -- https://docs.coincap.io/


In [1]:
api_key = "Paste your api-key"

In [2]:
import requests

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
import plotly.express as px

import time
from datetime import datetime

# 1.1 Анализ и поиск решения

Парсинг исторических данных

Для парсинга я выбрал API coincap.io
и временной отрезок от обновления ETH Gray Glacier (Jun 30, 2022) до последних имеющихся данных на момент парса (May 8,2023)

In [3]:


url ="https://api.coincap.io/v2/assets/bitcoin/history"
url2 ="https://api.coincap.io/v2/assets/ethereum/history"
params = {
    "interval":"h12",
    "start":1656586444000,
    "end":1683562141000
}
payload = {}
headers = {
  'Authorization': f'Bearer {api_key}'
}

response = requests.request("GET", url, headers=headers, data=payload,params=params)
print(response.status_code)
df_btc = pd.DataFrame(response.json()["data"])
time.sleep(1)
response = requests.request("GET", url2, headers=headers, data=payload,params=params)
print(response.status_code)
df_eth = pd.DataFrame(response.json()["data"])


200
200


После парсинга привел данные в один pandas dataframe, и исправил некоторые типы данных.

In [4]:
data = pd.DataFrame()
data["time_stamp"] = df_btc["time"].astype("int64")
data["date"] = [datetime.fromtimestamp(int(i)*0.001) for i in data["time_stamp"]]
data["BTC_USD"] = df_btc["priceUsd"].astype('float')
data["BTC_Supply"]=df_btc["circulatingSupply"].astype('float')
data["ETH_USD"] = df_eth["priceUsd"].astype('float')
data["ETH_Supply"]=df_eth["circulatingSupply"].astype('float')
data["BTC_USD_01"]= data["BTC_USD"]*0.1

Далее визуализировал зависимость на кросплоте ETH/BTC.

In [5]:

fig = px.scatter(data, y="ETH_USD", x="BTC_USD", trendline="ols")
fig.show()

Возможно я выбрал не совсем правильный временной интервал, для того чтоб это понять надо нанести на график время обновлений ETH.
Для этого обращаюсь к сайту https://ethereum.org/en/history/#gray-glacier


In [6]:
update_ETH = {
    1656586444000:"Gray_Glacier",
    1662464087000:"Bellatrix",
    1663224162000:"Paris",
    1681338455000:"Shanghai", 
}
data["ETH_UPDATE"]=None

In [7]:
for i in update_ETH.keys():
    data["ETH_UPDATE"][data["time_stamp"]>i]=update_ETH[i]

C:\Users\gaydu\AppData\Local\Temp\ipykernel_10672\2010646403.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:

fig = px.scatter(data, y="ETH_USD", x="BTC_USD",color="ETH_UPDATE", trendline="ols")
fig.show()

In [9]:

fig = px.scatter(data, y=["ETH_USD"], x="date",color="ETH_UPDATE", trendline="ols")
fig.show()

На графиках видна четкая зависимость между обновлениями ETH и законом изменения цены, однако использовать только 1 интервал невозможно изза не стабильности цены сразу после обновления. 
Остается несколько вариантов, но ради сокращения времени, и упращения работы, предлагаю строить тренд зависимости по данным за последние 3 месяца.


In [10]:
data_3m = data[data["time_stamp"]>(datetime.timestamp(datetime.utcnow())-7889229)*1000]

In [11]:

fig = px.scatter(data_3m, y="ETH_USD", x="BTC_USD",color="ETH_UPDATE", trendline="ols")
fig.show()

In [12]:
# Extract the Bitcoin and Ethereum prices
bitcoin_prices = np.array(data_3m['BTC_USD']).reshape((-1, 1)).astype(float)
ethereum_prices = np.array(data_3m['ETH_USD']).astype(float)

# Create a random forest regression model and fit it to the training data
model = LinearRegression()
model.fit(bitcoin_prices,ethereum_prices)
print(f"coef - {model.coef_[0]}\n",
      f"intercept - {model.intercept_}"
)

# Use the model to predict the Ethereum price based on the test data
y_pred = model.predict(bitcoin_prices)


# Evaluate the model's performance using R-squared and Mean Squared Error
r_squared = r2_score(ethereum_prices, y_pred)
mse = mean_squared_error(ethereum_prices, y_pred)

print(f'R-squared: {round(r_squared, 3)}, Mean Squared Error: {round(mse, 3)}')

coef - 0.05217199292699814
 intercept - 391.2927450618338
R-squared: 0.911, Mean Squared Error: 2338.525


Получаем очень хорошую зависимость r2 = 0.908, среднеквадратичная ошибка 2365.623 долларов.

In [13]:
data["ETH_LR"] = model.predict(np.array(data.BTC_USD).reshape((-1, 1)))
data["ETH_LR_CF"] = model.intercept_ + model.coef_[0]*data.BTC_USD

In [14]:
data[["ETH_LR","ETH_LR_CF"]].describe() 

,ETH_LR,ETH_LR_CF
count,619.000000,619.000000
mean,1525.175346,1525.175346
std,201.554041,201.554041
min,1213.296163,1213.296163
25%,1391.086176,1391.086176
50%,1494.913733,1494.913733
75%,1626.643665,1626.643665
max,2002.300533,2002.300533


In [15]:
fig = px.scatter(data, y=["ETH_USD","ETH_LR", "BTC_USD_01"], x="date", trendline="ols")
fig.show()

In [16]:
data["ETH_Varaity_percent"]= data["ETH_USD"]/data["ETH_LR"]*100-100

In [17]:
fig = px.scatter(data, y=["ETH_Varaity_percent"], x="date", trendline="ols")
fig.show()

На графике выше показано изменение ценны в процентном отношении

In [18]:
data["ETH_Untethered"]= data["ETH_USD"]-data["ETH_LR"]

In [19]:
fig = px.scatter(data, y=["ETH_Untethered"], x="date", trendline="ols")
fig.show()

* На графике выше показаны движения не вызванные влиянием цены BTCUSDT

Теперь когда мы получили предстовление о данных и нашли подходящую методику давайте создадим более точный и функциональный алгоритм решения данной задачи и напишем функции для второй части тестового задания.

# 1.2 Решение

Для решения первой задачи тестого я решил использовать линейный корреляционный метод.

* Собрать историческую информацию
* Построить линейную регрессию по зависимости BTC x ETH
* Посчитать стоимость ETH по линейной зависимости
* Разница между ценой ETH и расчитанной ценой показывает независимое движение цены ETH

In [20]:
def pars_data(coin_name_list,interval="h2"):
  """"valid intervals: m1, m5, m15, m30, h1, h2, h6, h12, d1"""
  
  time_now = datetime.timestamp(datetime.utcnow())*1000
  data = []

  for coin_name in coin_name_list:
    url = f"https://api.coincap.io/v2/assets/{coin_name}/history"

    params = {
        "interval":f"{interval}",
        "start":time_now-2629743000,
        "end":time_now
    }
    payload = {}
    headers = {
      'Authorization': f'Bearer {api_key}'
    }

    response = requests.request("GET", url, headers=headers, data=payload,params=params)
    print(response.status_code)
    df_tmp = pd.DataFrame(response.json()["data"])
    df_tmp["time"] = df_tmp["time"].astype("int64")
    df_tmp["date"] = [datetime.fromtimestamp(int(i)*0.001) for i in df_tmp["time"]]
    df_tmp["priceUsd"] = df_tmp["priceUsd"].astype('float')
    df_tmp["circulatingSupply"]=df_tmp["circulatingSupply"].astype('float')

    df_tmp = df_tmp.rename(columns={"priceUsd":f"{coin_name}_priceUSD","circulatingSupply":f"{coin_name}_Supply"})
    df_tmp.set_index("time",inplace=True)
    
    data.append(df_tmp)
    time.sleep(1)
  
  data = pd.concat(data,axis=1)
  data = data.T.drop_duplicates().T

  return data
data = pars_data(["bitcoin","ethereum"],"h2")

200
200


In [21]:
data.head(4)

,bitcoin_priceUSD,bitcoin_Supply,date,ethereum_priceUSD,ethereum_Supply
time,,,,,
1681171200000,29668.305957,19342687.0,2023-04-11 03:00:00,1913.278927,120457776.0
1681178400000,30205.321812,19342718.0,2023-04-11 05:00:00,1929.237561,120457776.0
1681185600000,30139.356959,19342831.0,2023-04-11 07:00:00,1926.556096,120457776.0
1681192800000,29938.404485,19342943.0,2023-04-11 09:00:00,1916.953419,120457776.0


In [22]:
fig = px.scatter(data, y="ethereum_priceUSD", x="bitcoin_priceUSD", trendline="ols")
fig.show()

In [23]:
def LinReg(x,y):
    bitcoin_prices = np.array(x).reshape((-1, 1)).astype(float)
    ethereum_prices = np.array(y).astype(float)

    # Create a random forest regression model and fit it to the training data
    model = LinearRegression()
    model.fit(bitcoin_prices,ethereum_prices)
    print(f"coef - {model.coef_[0]}\n",
        f"intercept - {model.intercept_}"
    )

    # Use the model to predict the Ethereum price based on the test data
    y_pred = model.predict(bitcoin_prices)

    # Evaluate the model's performance using R-squared and Mean Squared Error
    r_squared = r2_score(ethereum_prices, y_pred)
    mse = mean_squared_error(ethereum_prices, y_pred)

    print(f'R-squared: {round(r_squared, 3)}, Mean Squared Error: {round(mse, 3)}')
    return model

model = LinReg(x=data["bitcoin_priceUSD"],y=data["ethereum_priceUSD"])
data["ETH_LR"] = model.intercept_ + model.coef_[0]*data.bitcoin_priceUSD

coef - 0.06877419668066524
 intercept - -64.39942886814629
R-squared: 0.617, Mean Squared Error: 2944.666


In [24]:
data["BTC_USD_01"]= data["bitcoin_priceUSD"]*0.1
fig = px.scatter(data, y=["ethereum_priceUSD","ETH_LR", "BTC_USD_01"], x="date", trendline="ols")
fig.show()

In [25]:
data["ETH_Varaity_percent"]= data["ethereum_priceUSD"]/data["ETH_LR"]*100-100
data["ETH_dif_dollars"]= data["ethereum_priceUSD"]-data["ETH_LR"]

In [26]:
data

,bitcoin_priceUSD,bitcoin_Supply,date,ethereum_priceUSD,ethereum_Supply,ETH_LR,BTC_USD_01,ETH_Varaity_percent,ETH_dif_dollars
time,,,,,,,,,
1681171200000,29668.305957,19342687.0,2023-04-11 03:00:00,1913.278927,120457776.0,1976.01448,2966.830596,-3.174853,-62.735553
1681178400000,30205.321812,19342718.0,2023-04-11 05:00:00,1929.237561,120457776.0,2012.947314,3020.532181,-4.158567,-83.709753
1681185600000,30139.356959,19342831.0,2023-04-11 07:00:00,1926.556096,120457776.0,2008.410634,3013.935696,-4.075588,-81.854538
1681192800000,29938.404485,19342943.0,2023-04-11 09:00:00,1916.953419,120457776.0,1994.590289,2993.840448,-3.892372,-77.636871
1681200000000,30150.185445,19343018.0,2023-04-11 11:00:00,1925.534668,120457776.0,2009.155355,3015.018545,-4.161982,-83.620687
...,...,...,...,...,...,...,...,...,...
1683763200000,27651.07035,19369737.0,2023-05-11 03:00:00,1844.298189,120317700.84569,1837.280722,2765.107035,0.381949,7.017468
1683770400000,27595.548844,19369793.0,2023-05-11 05:00:00,1837.476786,120317700.84569,1833.462275,2759.554884,0.218958,4.014512
1683777600000,27481.595549,19369868.0,2023-05-11 07:00:00,1829.934824,120309683.353709,1825.625228,2748.159555,0.236061,4.309596


In [27]:
fig = px.scatter(data, y=["ETH_Varaity_percent"], x="date", trendline="ols",hover_data=["ETH_dif_dollars"])
fig.show()

На графике выше мы можем наблюдать собственное движение цены в процентах и при наведении на точку курсором видно изменение цены в доларах (ETH_dif_dollars)